In [2]:
import torch
import torch.nn as nn
import pandas as pd

In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS, PUCK_SQUARE_DISTANCES, PUCK_ANGLE_FEATURES
from isaac.sanity import class_proportions
from isaac.models import initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data_directory = "features_plots/"

In [4]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

create_directory(data_directory)

cuda:0


In [5]:
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 25
STEP_SIZE = 1
SEQ_END = 1800

In [6]:
ALL_COLS = BASIC_TRAINING_COLS + PUCK_SQUARE_DISTANCES + PUCK_ANGLE_FEATURES

In [7]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=ALL_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=500, cols=ALL_COLS)

100%|██████████| 10/10 [00:00<00:00, 126.13it/s]


# MASS TRAINING

In [8]:
INPUT_DIM = None    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

In [9]:
labels = ["distances+angles", "distances", "angles", "baseline"]
stats_dfs = []

for tr_features, label in zip([ALL_COLS, BASIC_TRAINING_COLS+PUCK_SQUARE_DISTANCES, 
                               BASIC_TRAINING_COLS+PUCK_ANGLE_FEATURES, BASIC_TRAINING_COLS], labels):
    
    loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=MASS_CLASS_COLS, 
                                      training_columns=tr_features, batch_size=BATCH_SIZE, 
                                      normalise_data=NORMALISE_DATA, device=device)
        
    network_params = (len(tr_features), HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["features", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_end=SEQ_END,
                                                                   step_size=STEP_SIZE)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["features"] = label
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
# stats.to_hdf(data_directory+"mass_stats.h5", key="stats")

100%|██████████| 10/10 [00:00<00:00, 845.06it/s]
Train_loss (1.07)	 Train_acc (60.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.41it/s] 
Train_loss (1.06)	 Train_acc (50.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.57it/s] 
Train_loss (1.05)	 Train_acc (60.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.49it/s] 
100%|██████████| 10/10 [00:00<00:00, 944.79it/s]
Train_loss (1.07)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.51it/s] 
Train_loss (1.16)	 Train_acc (40.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.57it/s] 
Train_loss (1.07)	 Train_acc (80.00)	 Val_acc (20.00): 100%|██████████| 2/2 [00:01<00:00,  1.57it/s] 
100%|██████████| 10/10 [00:00<00:00, 908.96it/s]
Train_loss (1.13)	 Train_acc (40.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.58it/s] 
Train_loss (1.07)	 Train_acc (80.00)	 Val_acc (20.00): 100%|██████████| 2/2 [00:01<00:00,  1.54it/s] 
Train_loss (1.09)	 Train_acc (50.00)	

# FORCE TRAINING

In [10]:
INPUT_DIM = None    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

In [ ]:
labels = ["distances+angles", "distances", "angles", "baseline"]
stats_dfs = []

for tr_features, label in zip([ALL_COLS, BASIC_TRAINING_COLS+PUCK_SQUARE_DISTANCES, 
                               BASIC_TRAINING_COLS+PUCK_ANGLE_FEATURES, BASIC_TRAINING_COLS], labels):
    
    loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=FORCE_CLASS_COLS, 
                                      training_columns=tr_features, batch_size=BATCH_SIZE, 
                                      normalise_data=NORMALISE_DATA, device=device)
        
    network_params = (len(tr_features), HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["features", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_end=SEQ_END,
                                                                   step_size=STEP_SIZE)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["features"] = label
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
# stats.to_hdf(data_directory+"force_stats.h5", key="stats")

100%|██████████| 10/10 [00:00<00:00, 1022.35it/s]
Train_loss (1.10)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.54it/s] 
Train_loss (1.11)	 Train_acc (70.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.54it/s] 
Train_loss (1.10)	 Train_acc (40.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.52it/s] 
100%|██████████| 10/10 [00:00<00:00, 1061.80it/s]
Train_loss (-1.00)	 Train_acc (30.00)	 Val_acc (40.00):   0%|          | 0/2 [00:00<?, ?it/s]